In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from datasets import load_dataset
model_name_or_path = 'mrm8488/HindiBERTa'
# model_name_or_path = 'ai4bharat/indic-bert'

In [14]:
raw_datasets = load_dataset('indic_glue', 'iitp-mr.hi')

Reusing dataset indic_glue (/home/rtx/.cache/huggingface/datasets/indic_glue/iitp-mr.hi/1.0.0/a0b7d1714992c5418c861bc2c1874217550020774e9f6576f52a02b17073380c)


In [15]:
num_labels = raw_datasets['train'].features['label'].num_classes
num_labels

3

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/rtx/.cache/huggingface/datasets/indic_glue/iitp-mr.hi/1.0.0/a0b7d1714992c5418c861bc2c1874217550020774e9f6576f52a02b17073380c/cache-0bf82e80472ea9a4.arrow


Loading cached processed dataset at /home/rtx/.cache/huggingface/datasets/indic_glue/iitp-mr.hi/1.0.0/a0b7d1714992c5418c861bc2c1874217550020774e9f6576f52a02b17073380c/cache-0b214f73c60e79c4.arrow


In [18]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["validation"]

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, num_labels=num_labels)

Some weights of the model checkpoint at mrm8488/HindiBERTa were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mrm8488/HindiBERTa and are newly initialized: ['

In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

In [21]:
from transformers import Trainer

import numpy as np
from datasets import load_metric

# metric = load_metric("accuracy")
metric = load_metric('indic_glue', 'iitp-mr')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset, compute_metrics=compute_metrics
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.039737,0.522581
2,0.935300,1.060425,0.532258
3,0.935300,1.209347,0.548387


TrainOutput(global_step=930, training_loss=0.7796611826906922, metrics={'train_runtime': 150.6731, 'train_samples_per_second': 6.172, 'total_flos': 881906575196160.0, 'epoch': 3.0})

In [23]:
trainer.evaluate()

{'eval_loss': 1.209346890449524,
 'eval_accuracy': 0.5483870967741935,
 'eval_runtime': 1.9608,
 'eval_samples_per_second': 158.096,
 'epoch': 3.0}